In [1]:
live = False

In [2]:
import skimage
import numpy as np
if live:
    from picamera2 import Picamera2
    import RPi.GPIO as GPIO

# Initial setup

In [3]:
# Set up camera
if live:
    picam2 = Picamera2()
    # Configure a simple capture mode
    config = picam2.create_still_configuration()
    picam2.configure(config)
    picam2.start()

In [4]:
# Set up servos
if live:
    # Motor A
    AIN1 = 6
    AIN2 = 13
    # Motor B
    BIN1 = 19
    BIN2 = 26
    # Frequency for PWM
    FREQ = 1000
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(AIN1, GPIO.OUT)
    GPIO.setup(AIN2, GPIO.OUT)
    GPIO.setup(BIN1, GPIO.OUT)
    GPIO.setup(BIN2, GPIO.OUT)
    pwm_AIN1 = GPIO.PWM(AIN1, FREQ)
    pwm_AIN2 = GPIO.PWM(AIN2, FREQ)
    pwm_BIN1 = GPIO.PWM(BIN1, FREQ)
    pwm_BIN2 = GPIO.PWM(BIN2, FREQ)
    pwm_AIN1.start(0)
    pwm_AIN2.start(0)
    pwm_BIN1.start(0)
    pwm_BIN2.start(0)

In [5]:
# Good to have
def relu(x):
    return np.maximum(x, 0)

In [6]:
# Load is_sign weights
model_dir = "models/is_sign"
is_M1 = np.loadtxt(f"{model_dir}/M1.csv", delimiter = ',')
is_M2 = np.loadtxt(f"{model_dir}/M2.csv", delimiter = ',')
is_M3 = np.loadtxt(f"{model_dir}/M3.csv", delimiter = ',')
is_b1 = np.loadtxt(f"{model_dir}/b1.csv", delimiter = ',')
is_b2 = np.loadtxt(f"{model_dir}/b2.csv", delimiter = ',')
is_b3 = np.loadtxt(f"{model_dir}/b3.csv", delimiter = ',')

In [7]:
# Load sign recognition weights
model_dir = "models/rec_sign"
rec_M1 = np.loadtxt(f"{model_dir}/M1.csv", delimiter = ',')
rec_M2 = np.loadtxt(f"{model_dir}/M2.csv", delimiter = ',')
rec_M3 = np.loadtxt(f"{model_dir}/M3.csv", delimiter = ',')
rec_b1 = np.loadtxt(f"{model_dir}/b1.csv", delimiter = ',')
rec_b2 = np.loadtxt(f"{model_dir}/b2.csv", delimiter = ',')
rec_b3 = np.loadtxt(f"{model_dir}/b3.csv", delimiter = ',')

# Image processing

## Take image and preprocess

In [8]:
def getDistance(bbox):
    width = bbox[3] - bbox[1]

    # Magic number (obtained with regression)
    a = 8108.02942543

    # Reasonable output range
    lower = 5.0
    upper = 200

    res = a/x

    if res < lower:
        return lower
    elif res > upper:
        return upper
    else:
        return res

In [9]:
# Get angle to found sign
def getAngle(centroid):
    x = centroid[1]
    # Magic numbers (obtained with regression)
    a = -2.87824117e-04
    b =  4.43471415e-01

    return a*x + b

In [10]:
def take_img(threshold = 0.85, sign_color = [1.1467, 51.18945, 135.2949]):
    img = picam2.capture_array()
    img = 1 - (np.linalg.norm(img - sign_color, axis = 2) / (255*np.sqrt(3)))
    img = skimage.transform.rotate(img, 180)
    binary = img > threshold
    labels = skimage.measure.label(binary)
    binary = None # Garbage collect unneeded array

    
    max_area = 1000
    sign = None
    distance = None
    angle = None

    for region in skimage.measure.regionprops(labels):
        if region.area > max_area:
            max_area = region.area
            (min_row, min_col, max_row, max_col) = region.bbox
            distance = getDistance(region.bbox)
            sign = img[min_row:max_row, min_col:max_col]
            angle = getAngle(region.centroid)

    distance = getDistance(bbox)
    
    return distance, angle, sign

In [11]:
# Pre-trained multilayer perceptron to distringuish whether we're looking at a sign
def is_sign(gray):
    state = skimage.transform.resize(gray, (32, 32)).flatten()
    state = relu(np.matmul(state, is_M1) + is_b1)
    state = relu(np.matmul(state, is_M2) + is_b2)
    state = np.matmul(state, is_M3) + is_b3
    return state > 0

In [12]:
# Pre-trained multilayer perceptron to distringuish between signs
def rec_sign(gray):
    state = skimage.transform.resize(gray, (32, 32)).flatten()
    state = relu(np.matmul(state, rec_M1) + rec_b1)
    state = relu(np.matmul(state, rec_M2) + rec_b2)
    state = relu(np.matmul(state, rec_M3) + rec_b3)
    return np.argmax(state) # 0 = back, 1 = left, 2 = right

In [13]:
if not live:
    back = skimage.io.imread("data/back_cropped/10.png")
    left = skimage.io.imread("data/left_cropped/10.png")
    right = skimage.io.imread("data/right_cropped/10.png")
    skimage.io.imread
    print(rec_sign(right))
    print(is_sign(right))

2
True


# Movement

In [14]:
# Get required turn time 
def getTime(tgt_angle):
    x = np.abs(tgt_angle)

    # Magic numbers (obtained with regression)
    a = 0.11596371
    b = 0.13241551

    # Use a linear + squareroot term to model the non-zero intercept
    # while still going smoothly to 0
    t = a*x + b*np.sqrt(x)
    
    # Return time and direction (True = right, False = left)
    return t, tgt_angle <= 0

In [15]:
def turn(tgt_angle):
    t, direction = getTime(tgt_angle)
    # TODO: implement

In [16]:
def left():
    turn(np.pi/2)

def right():
    turn(-np.pi/2)

def back():
    turn(np.pi)

In [17]:
def forward():
    # TODO: implement
    return None

In [18]:
def perturb():
    turn(np.random.normal(0.05, 0.4))

# Control loop

In [19]:
def control_loop():
    distance, angle, sign = take_img()

    # Did we find anything at all?
    if distance is None or angle is None or sign is None:
        print("No blob found in image, perturbing!")
        perturb()
        return 1

    # Did we find a sign?
    if not is_sign(sign):
        print("Found blob is not a sign, perturbing!")
        perturb()
        return 2

    # Is the sign centered?
    centered_threshold = 0.2
    if np.abs(angle) >= centered_threshold:
        print("Sign is not centered, turning towards it.")
        turn(angle)
        return 3

    # Are we close to the sign?
    distance_threshold = 10
    if distance >= distance_threshold:
        print("Sign is far away, moving towards it.")
        forward()
        return 4

    # Recognize sign
    instruction = rec_sign(sign)
    if instruction == 0:
        print("Back sign recognised, turning.")
        back()
        return 5
    elif instruction == 1:
        print("Left sign recognised, turning.")
        left()
        return 6
    else:
        print("Right sign recognised, turning.")
        right()
        return 7

In [20]:
l = 5000
for i in range(l):
    control_loop()

NameError: name 'picam2' is not defined